In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18
from sklearn.metrics import accuracy_score
from PIL import Image

In [15]:
# Custom Dataset Class
class EyesDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['noneyes', 'eyes']
        self.data = []

        for i, cls in enumerate(self.classes):
            class_dir = os.path.join(root_dir, cls)
            for filename in os.listdir(class_dir):
                filepath = os.path.join(class_dir, filename)
                self.data.append((filepath, i))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

In [21]:
# Define a simpler model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 112 * 112, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [23]:
# Create datasets and dataloaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [24]:
# Create datasets and dataloaders
train_dataset = EyesDataset(root_dir=r'C:\Users\Ayesha\Desktop\Object Detection using Pytorch\Linear Regression using pytorch\dataset\train', transform=transform)
val_dataset = EyesDataset(root_dir=r'C:\Users\Ayesha\Desktop\Object Detection using Pytorch\Linear Regression using pytorch\dataset\val', transform=transform)

In [29]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

In [26]:
# Initialize the model
num_classes = len(train_dataset.classes)
model = SimpleCNN(num_classes=num_classes)

In [27]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [30]:
# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [31]:
# Validation loop
model.eval()
all_preds = []
all_labels = []

In [33]:
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())


In [34]:
# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Validation Accuracy: {accuracy:.4f}')

Validation Accuracy: 0.9825


In [36]:
def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img = transform(img)
    img = img.unsqueeze(0)  
    return img

In [37]:
def predict_image(model, image_path):
    img = preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(img)
    _, predicted_class = torch.max(outputs, 1)
    return predicted_class.item()


In [41]:
sample_image_path = r'C:\Users\Ayesha\Desktop\Object Detection using Pytorch\Linear Regression using pytorch\eyes-test.jpg'
predicted_class = predict_image(model, sample_image_path)

if predicted_class == 0:
    print("Predicted class: Non-eye")
else:
    print("Predicted class: Eye")


Predicted class: Eye


In [47]:
sample_image_path = r'C:\Users\Ayesha\Desktop\Object Detection using Pytorch\Linear Regression using pytorch\dataset\val\noneyes\025.png'
predicted_class = predict_image(model, sample_image_path)

if predicted_class == 0:
    print("Predicted class: Non-eye")
else:
    print("Predicted class: Eye")


Predicted class: Non-eye
